<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/Copie_de_SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install all requirements (run once)

# System
import locale, os
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade transformers==4.31.0 torch accelerate==0.21.0 bitsandbytes==0.41.0 tensorflow==2.13.0 onnxruntime-gpu==1.15.1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Application
!pip install nltk
!pip install pytorch-crf==0.7.2 youtube_transcript_api==0.6.1 fastpunct==2.0.2 nnsplit==0.5.9.post0 tiktoken==0.4.0
import nltk
nltk.download('punkt')
!git clone https://github.com/xashru/punctuation-restoration.git
!gdown https://drive.google.com/u/0/uc?id=17BPcnHVhpQlsOTC8LEayIFFJ7WkL00cr&export=download

# Youtube processing

In [ ]:
#@title Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=jQL0ZeHtXFc" #@param {type:"string"}

In [ ]:
#@title Extract the youtube video transcript

def YTVideoToText(video_link):
  # installing & importing libraries
  from youtube_transcript_api import YouTubeTranscriptApi

  # fetching video transcript
  video_id = video_link.split("=")[1]
  transcript = YouTubeTranscriptApi.get_transcript(video_id)

  # iterating throughout and adding all text together (recovering ponctuations)
  flat_transcript = ""
  for tu in transcript:
      flat_transcript += ' ' + tu['text']

  return flat_transcript

video_full_text = YTVideoToText(video_link)

In [ ]:
#@title Sentence splitter
max_tokens_per_sentence = 416 #@param {type:"slider", min:64, max:8192, step:32}

import tiktoken
from nnsplit import NNSplit
splitter = NNSplit.load("en")
encoding = tiktoken.encoding_for_model("gpt2")

# returns `Split` objects
video_sentences = []
current_sentence = ""
for sentence in (splitter.split([video_full_text])[0]):
  if (len(encoding.encode(f"{current_sentence}")) < max_tokens_per_sentence):
    current_sentence += f"{sentence}"
  else:
    video_sentences.append(current_sentence)
    current_sentence = ""


In [ ]:
# @title Extract the sentences
import spacy

def process_text(full_text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(full_text)
    return [f'{sentence}' for sentence in doc.sents]

# Utilisation de la fonction
video_sentences = process_text(video_full_text)
video_sentences


In [ ]:
#@title Restore video transcript punctuation
spell_check = True #@param {type:"boolean"}

def FlatTextToPunct(txt):
  # installing & importing libraries
  from fastpunct import FastPunct

  # The default language is 'english'
  fastpunct = FastPunct()

  return fastpunct.punct(txt, correct= spell_check)

#video_restored_text = FlatTextToPunct(video_sentences)



In [ ]:
with open("video_full_text", 'w', encoding='utf-8') as f:
  f.write(video_full_text)

In [ ]:
%%shell

python punctuation-restoration/src/inference.py --pretrained-model=roberta-large --weight-path=roberta-large-en.pt --language=en
--in-file=video_full_text --out-file=video_restored_text

In [ ]:
with open("video_restored_text", 'r', encoding='utf-8') as f:
    video_restored_text = f.read()

# Summarization

In [ ]:
#@title Load the model into memory
from transformers import pipeline

model_id = "facebook/bart-large-cnn" #@param {type:"string"}
summarizeryt = pipeline('summarization', model= model_id, device=0)

In [ ]:
#@title Join sentences in paragraphs of less the specified token size
summarize_token_width = 500 #@param {type:"slider", min:250, max:2048, step:250}

In [ ]:

def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text.append(' ').append(out)

  # returning summary
  return summarized_text;

In [ ]:
#@title Show the transcript of the video
video_full_text

In [ ]:
#@title Show the summarized text of the video
sum_text